# Entraînement d'un Modèle de Computer Vision YOLO Personnalisé pour Dofus (Transfer Learning)
## INTRODUCTION
Bienvenue dans ce guide interactif ! L'objectif est de vous accompagner, étape par étape, dans l'entraînement d'un modèle de détection d'objets YOLO personnalisé. Grâce à ce modèle, vous pourrez reconnaître automatiquement des ressources pour bucheron et paysan dans des images du jeu Dofus.

**Les outils que nous allons utiliser :**
1.  **Votre dataset Dofus annoté :**
    *   **Option A (Label Studio) :** Vous l'avez préparé avec soin en utilisant Label Studio.
    *   **Option B (Roboflow) :** Vous pouvez utilisé roboflow pour récupérer mon dataset ou faire le votre.
2.  **Ultralytics :** Une bibliothèque Python puissante et conviviale pour entraîner les dernières générations de modèles YOLO.
3.  **Google Colab :** Notre environnement de travail. Il nous offre un accès gratuit à des GPUs (processeurs graphiques), indispensables pour accélérer l'entraînement.

**Ce que vous obtiendrez à la fin :**
-   Un modèle YOLO entraîné et performant (un fichier `.pt`).
-   Un fichier ZIP (`dofus_AI.zip`) contenant votre modèle et les fichiers de configuration, prêt à être téléchargé et utilisé dans vos propres projets.

Suivez attentivement les étapes ci-dessous. Chaque bloc de code est exécutable directement dans Colab.


# ÉTAPE 0 : VÉRIFIER LA DISPONIBILITÉ DU GPU NVIDIA
Pour que l'entraînement soit rapide, il est crucial d'utiliser un GPU.
**Action :** Assurez-vous que Colab utilise bien un GPU.
1.  Dans le menu Colab : "Exécution" (ou "Runtime").
2.  Cliquez sur "Modifier le type d'exécution" (ou "Change runtime type").
3.  Dans la section "Accélérateur matériel" (ou "Hardware accelerator"), sélectionnez "GPU" (généralement T4).
4.  Sauvegardez.

Exécutez la commande ci-dessous pour confirmer que le GPU est bien détecté :

In [ ]:
!nvidia-smi

*(Vous devriez voir apparaître des informations sur un GPU NVIDIA, comme Tesla T4, K80, etc.)*

---

**SI VOUS UTILISEZ UN DATASET EXPORTÉ DE LABEL STUDIO, SUIVEZ L'ÉTAPE 1 CI-DESSOUS.**   
**SI VOUS UTILISEZ UN DATASET EXPORTÉ DE ROBOFLOW, PASSEZ DIRECTEMENT À L'ÉTAPE 1 (ALTERNATIVE) CI-DESSOUS.**

---

## ÉTAPE 1 : PRÉPARATION DE VOTRE DATASET DOFUS (EXPORTÉ DE LABEL STUDIO)

Cette étape est fondamentale : la qualité de votre modèle dépendra directement de la qualité de votre dataset.

### 1.1 Rappels sur la préparation de votre dataset avec Label Studio
Pour créer le dataset j'ai utilisé Label Studio ([https://labelstud.io/](https://labelstud.io/)) qui permet d'annoter nos images Dofus. Pour la détection d'objets avec YOLO, le processus consiste à :
1.  Dessiner des **boîtes englobantes** (bounding boxes) autour des objets d'intérêt, des ressources dans notre cas.
2.  Assigner une **classe** (un nom) à chaque boîte (par exemple, "Frene", "Ble", "Eau").

**Format d'exportation depuis Label Studio :**
Lorsque vous exportez votre projet depuis Label Studio, il est impératif de choisir le format **"YOLO"**.
Cet export génère typiquement :
-   Un dossier `images/` : Contient toutes vos images originales (ex: `.jpg`, `.png`).
-   Un dossier `labels/` : Contient des fichiers `.txt` portant le même nom que vos images. Chaque fichier `.txt` décrit les objets de l'image correspondante, avec une ligne par objet :
    `<class_id> <x_center_norm> <y_center_norm> <width_norm> <height_norm>`
    *   `<class_id>` : Un numéro identifiant la classe (commençant à 0).
    *   Les autres valeurs sont les coordonnées normalisées (entre 0 et 1) du centre de la boîte, sa largeur et sa hauteur, relatives à la taille de l'image.
-   Un fichier `classes.txt` : Liste tous les noms de vos classes, un par ligne. L'ordre est CRUCIAL car il définit les `class_id` (la première ligne est la classe 0, la deuxième la classe 1, etc.).

**✅ Action requise de votre part (préparation du ZIP) :**
1.  Exportez votre projet Label Studio au format YOLO.
2.  Créez un fichier ZIP unique nommé `data.zip`.
3.  **Important :** Ce `data.zip` DOIT contenir **À SA RACINE** (pas dans un sous-dossier à l'intérieur du ZIP) :
    -   Le dossier `images/` (avec vos images).
    -   Le dossier `labels/` (avec vos fichiers d'annotation `.txt`).
    -   Le fichier `classes.txt` (listant vos classes).

**Exemple de structure attendue à l'intérieur de `data.zip` :**
```
data.zip/
  ├── images/
  │   ├── dofus_image1.jpg
  │   ├── dofus_image2.png
  │   └── ...
  ├── labels/
  │   ├── dofus_image1.txt
  │   ├── dofus_image2.txt
  │   └── ...
  └── classes.txt
```
**Exemple de contenu du fichier `classes.txt` pour Dofus :**
```
Frene
Ble
Eau
...
```
*(Assurez-vous que les noms de classes sont exacts et dans l'ordre souhaité.)*

### 1.2 Téléverser votre `data.zip` sur Colab

Vous avez deux options pour importer votre `data.zip` dans l'environnement Colab.

**OPTION A : Téléchargement direct via l'interface Colab (pour les petits fichiers, < 100Mo)**
1.  Dans la barre latérale gauche de Colab, cliquez sur l'icône "Dossier" 📁.
2.  Cliquez sur l'icône "Importer dans l'espace de stockage de la session" (📄↑).
3.  Sélectionnez votre fichier `data.zip` depuis votre ordinateur.
4.  Attendez que le téléchargement soit complet (le fichier `data.zip` apparaîtra dans la liste).

**OPTION B : Copier depuis Google Drive (recommandé pour les datasets plus volumineux)**
1.  Uploadez votre `data.zip` sur votre Google Drive (par exemple, dans un dossier `MesProjetsIA/Dofus`).  (Par exemple dans mon cas j'ai créé un dossier dans mon Drive appelé DOFUS_BOT où j'ai mis mon `data.zip`)
2.  Exécutez le code ci-dessous. Vous devrez autoriser Colab à accéder à votre Drive.
3.  **⚠️ ATTENTION :** Modifiez la ligne `!cp /content/gdrive/MyDrive/DOFUS_BOT/data.zip ...` pour qu'elle corresponde au chemin exact de votre `data.zip` sur votre Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/DOFUS_BOT/data.zip /content/data.zip # Adaptez ce chemin !

### 1.3 Décompresser le dataset et organiser les données pour l'entraînement
Une fois `data.zip` uploadé, il devrait se trouver dans `/content/data.zip`.
Vérifions sa présence :

In [ ]:
!ls -lh /content/data.zip

Maintenant, décompressons ce ZIP. Nous allons placer son contenu dans un dossier `/content/custom_data`.
Ce dossier `custom_data` contiendra donc vos dossiers `images/`, `labels/` et votre fichier `classes.txt` (exactement la structure que vous aviez dans le ZIP).

In [ ]:
!unzip -q /content/data.zip -d /content/custom_data
# Vérifions le contenu de custom_data
!ls /content/custom_data

*(Vous devriez voir `images`, `labels`, et `classes.txt` listés.)*

Ultralytics (et les bonnes pratiques en Machine Learning) nécessite que les données soient séparées en au moins deux ensembles :
-   **Ensemble d'entraînement (train) :** Utilisé pour apprendre au modèle.
-   **Ensemble de validation (val) :** Utilisé pour évaluer la performance du modèle sur des données qu'il n'a jamais vues pendant l'entraînement. Cela aide à détecter le sur-apprentissage (overfitting) et à choisir le "meilleur" modèle.

Le script Python ci-dessous va :
1.  Prendre vos données depuis `/content/custom_data/`.
2.  Créer une nouvelle structure de dossiers dans `/content/data/` avec des sous-dossiers `train/images`, `train/labels`, `validation/images`, `validation/labels`.
3.  Répartir aléatoirement vos images (et leurs labels correspondants) entre ces ensembles d'entraînement et de validation. Nous utiliserons une répartition classique de 80% pour l'entraînement et 20% pour la validation.

In [ ]:
import os
import shutil
import random
from pathlib import Path

# --- Configuration du split ---
data_path_source = "/content/custom_data"  # Dossier source après décompression de data.zip
output_data_root = "/content/data"         # Dossier de sortie pour les données organisées
train_percentage_split = 0.8               # 80% pour l'entraînement, 20% pour la validation

print(f"Source des données : {data_path_source}")
print(f"Destination des données organisées : {output_data_root}")
print(f"Pourcentage pour l'entraînement : {train_percentage_split*100}%\n")

# Vérifier si le dossier source existe
if not os.path.isdir(data_path_source):
   print(f"ERREUR : Le dossier source '{data_path_source}' est introuvable.")
   print("Veuillez vérifier que votre `data.zip` a été correctement décompressé dans `custom_data`.")
   # Vous pouvez ajouter un `raise Exception(...)` ici si vous voulez arrêter le script
else:
    # Définir les chemins des sous-dossiers source
    input_image_path_str = os.path.join(data_path_source, 'images')
    input_label_path_str = os.path.join(data_path_source, 'labels')

    # Vérifier l'existence des dossiers images et labels source
    if not os.path.isdir(input_image_path_str):
        print(f"ERREUR : Le dossier d'images source '{input_image_path_str}' est introuvable dans '{data_path_source}'.")
    if not os.path.isdir(input_label_path_str):
        print(f"ERREUR : Le dossier de labels source '{input_label_path_str}' est introuvable dans '{data_path_source}'.")

    # Définir les chemins des dossiers de destination
    train_img_path_str = os.path.join(output_data_root, 'train/images')
    train_txt_path_str = os.path.join(output_data_root, 'train/labels')
    val_img_path_str = os.path.join(output_data_root, 'validation/images')
    val_txt_path_str = os.path.join(output_data_root, 'validation/labels')

    # Créer les dossiers de destination s'ils n'existent pas
    for dir_path_str in [train_img_path_str, train_txt_path_str, val_img_path_str, val_txt_path_str]:
       if not os.path.exists(dir_path_str):
          os.makedirs(dir_path_str)
          print(f"Dossier créé : {dir_path_str}")

    # Obtenir la liste de tous les fichiers images
    # Utilisation de Path pour rglob qui cherche récursivement (utile si vous avez des sous-dossiers dans images/)
    # Si vos images sont toutes à la racine de 'images/', os.listdir pourrait aussi fonctionner.
    img_file_list = [p for p in Path(input_image_path_str).rglob('*') if p.is_file()]
    # S'assurer que ce sont bien des fichiers et pas des dossiers cachés comme .DS_Store
    img_file_list = [f for f in img_file_list if not f.name.startswith('.')]


    if not img_file_list:
        print(f"ERREUR : Aucun fichier image trouvé dans '{input_image_path_str}'.")
    else:
        print(f"\nNombre total de fichiers images trouvés : {len(img_file_list)}")

        # Mélanger la liste des fichiers pour une sélection aléatoire
        random.shuffle(img_file_list)

        # Déterminer le nombre de fichiers pour chaque ensemble
        file_num_total = len(img_file_list)
        train_num = int(file_num_total * train_percentage_split)
        val_num = file_num_total - train_num

        print(f"Nombre d'images pour l'entraînement : {train_num}")
        print(f"Nombre d'images pour la validation : {val_num}\n")

        # Répartir les fichiers
        print("Début de la copie des fichiers d'entraînement...")
        for i in range(train_num):
            img_path_obj = img_file_list[i]
            img_filename = img_path_obj.name
            base_filename = img_path_obj.stem
            label_filename = base_filename + '.txt'
            label_path_source_obj = Path(input_label_path_str) / label_filename

            # Copier l'image
            shutil.copy(str(img_path_obj), os.path.join(train_img_path_str, img_filename))

            # Copier le fichier label correspondant s'il existe
            if label_path_source_obj.exists():
                shutil.copy(str(label_path_source_obj), os.path.join(train_txt_path_str, label_filename))
            # else:
            #     print(f"  INFO: Fichier label '{label_filename}' non trouvé pour l'image d'entraînement '{img_filename}'. (Peut être une image de fond)")

        print("Fin de la copie des fichiers d'entraînement.")
        print("Début de la copie des fichiers de validation...")
        for i in range(train_num, file_num_total): # Les images restantes vont à la validation
            img_path_obj = img_file_list[i]
            img_filename = img_path_obj.name
            base_filename = img_path_obj.stem
            label_filename = base_filename + '.txt'
            label_path_source_obj = Path(input_label_path_str) / label_filename

            # Copier l'image
            shutil.copy(str(img_path_obj), os.path.join(val_img_path_str, img_filename))

            # Copier le fichier label correspondant s'il existe
            if label_path_source_obj.exists():
                shutil.copy(str(label_path_source_obj), os.path.join(val_txt_path_str, label_filename))
            # else:
            #     print(f"  INFO: Fichier label '{label_filename}' non trouvé pour l'image de validation '{img_filename}'. (Peut être une image de fond)")
        print("Fin de la copie des fichiers de validation.")
        print("\nRépartition des données terminée !")

# Vérifions la structure créée dans /content/data (facultatif mais utile)
print("\nContenu de /content/data :")
!ls /content/data
print("\nContenu de /content/data/train/images (premiers fichiers) :")
!ls /content/data/train/images | head -n 5
print("\nContenu de /content/data/train/labels (premiers fichiers) :")
!ls /content/data/train/labels | head -n 5
print("\nContenu de /content/data/validation/images (premiers fichiers) :")
!ls /content/data/validation/images | head -n 5
print("\nContenu de /content/data/validation/labels (premiers fichiers) :")
!ls /content/data/validation/labels | head -n 5
print("--- Fin de la vérification ---")

YOLO a besoin d'un fichier de configuration, traditionnellement nommé `data.yaml`. Ce fichier est comme une "carte d'identité" de votre dataset pour le modèle. Il lui indique :
-   Où trouver les images d'entraînement et de validation.
-   Le nombre de classes d'objets à détecter.
-   Les noms de ces classes.

Nous allons générer ce fichier `data.yaml` automatiquement à partir de votre fichier `classes.txt` (que vous avez fourni dans `data.zip` et qui se trouve maintenant dans `/content/custom_data/classes.txt`).

In [ ]:
import yaml
import os

def create_data_yaml(path_to_classes_txt_in_custom_data, output_data_yaml_path, root_data_dir_for_yaml):
  """
  Crée le fichier data.yaml nécessaire pour l'entraînement YOLO.

  Args:
    path_to_classes_txt_in_custom_data (str): Chemin vers votre fichier classes.txt original (ex: /content/custom_data/classes.txt).
    output_data_yaml_path (str): Chemin où le fichier data.yaml sera sauvegardé (ex: /content/dofus_data.yaml).
    root_data_dir_for_yaml (str): Chemin racine que YOLO utilisera pour trouver les dossiers train/val (ex: /content/data).
  """
  if not os.path.exists(path_to_classes_txt_in_custom_data):
    print(f"ERREUR : Le fichier {path_to_classes_txt_in_custom_data} est introuvable !")
    print("Veuillez vérifier que votre `data.zip` contenait bien `classes.txt` et qu'il a été extrait correctement.")
    return

  with open(path_to_classes_txt_in_custom_data, 'r') as f:
    classes = [line.strip() for line in f if line.strip()] # Lire les classes, ignorer les lignes vides

  if not classes:
    print(f"ERREUR : Aucune classe n'a été trouvée dans {path_to_classes_txt_in_custom_data}. Le fichier est-il vide ou mal formaté ?")
    return

  number_of_classes = len(classes)
  print(f"Nombre de classes détectées : {number_of_classes}")
  print(f"Noms des classes : {classes}")

  data_config = {
      'path': root_data_dir_for_yaml,  # Chemin absolu vers le dossier contenant train/ et validation/
      'train': 'train/images',       # Chemin relatif à 'path' pour les images d'entraînement
      'val': 'validation/images',    # Chemin relatif à 'path' pour les images de validation
      # 'test': Optional: chemin vers les images de test
      'nc': number_of_classes,
      'names': classes
  }

  with open(output_data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, sort_keys=False, indent=2)
  print(f"Fichier de configuration YAML créé avec succès : {output_data_yaml_path}")

# Définition des chemins
# Le fichier classes.txt original se trouve dans custom_data après décompression.
path_to_original_classes_txt = '/content/custom_data/classes.txt'
# Le dossier racine pour les données formatées (train/val) est /content/data.
data_root_for_yolo = '/content/data'
# Nous allons créer notre fichier de configuration data.yaml à la racine de /content.
path_to_output_yaml = '/content/dofus_data.yaml' # Nommez-le comme vous voulez

create_data_yaml(path_to_original_classes_txt, path_to_output_yaml, data_root_for_yolo)

# Affichons le contenu du fichier YAML créé pour vérification
print('\nContenu du fichier data.yaml généré :\n')
!cat {path_to_output_yaml}

## ÉTAPE 1 (ALTERNATIVE) : PRÉPARATION DE VOTRE DATASET DOFUS (EXPORTÉ DE ROBOFLOW)

Mon dataset est disponible sur roboflow pour le télécharger ([https://roboflow.com/](https://universe.roboflow.com/mathisl-qvljq/dofus-resource-detection/dataset/1))

### 1.A Exporter votre dataset depuis Roboflow
1.  Dans votre projet Roboflow, après avoir annoté et généré une version de votre dataset.
2.  Cliquez sur "Export Dataset".
3.  Choisissez le format **"YOLOv11"** . Ce format inclut typiquement :
    *   Des dossiers `train/`, `valid/` (et parfois `test/`) contenant chacun des sous-dossiers `images/` et `labels/`.
    *   Un fichier `data.yaml` déjà configuré pour votre dataset.
4.  Choisissez l'option "download zip to computer". Vous obtiendrez un fichier ZIP (par exemple, `MonProjetDofus.zip` ou `Dofus-Resources-1.zip`).
5.  Renommer le `data.zip`.

### 1.B Téléverser votre `data.zip` (exporté de Roboflow) sur Colab

Comme pour l'option Label Studio, vous pouvez soit le télécharger directement, soit passer par Google Drive (recommandé).

1.  Uploadez le fichier ZIP téléchargé de Roboflow sur votre Google Drive (par exemple, dans un dossier `MesProjetsIA/DOFUS_BOT`).
2.  Exécutez le code ci-dessous. Vous devrez autoriser Colab à accéder à votre Drive.
3.  **⚠️ ATTENTION :**
    * Renommer le ZIP en `data.zip`.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# MODIFIER CETTE LIGNE AVEC LE CHEMIN DE VOTRE FICHIER ZIP SUR GOOGLE DRIVE
path_to_zip_on_drive = f"/content/gdrive/MyDrive/DOFUS_BOT/data.zip" # EXEMPLE: "/content/gdrive/MyDrive/MesDatasets/Roboflow_Exports/data.zip"
# --- FIN DES MODIFICATIONS ---

destination_zip_path_colab = "/content/data.zip"

print(f"Tentative de copie de {path_to_zip_on_drive} vers {destination_zip_path_colab}")
!cp "{path_to_roboflow_zip_on_drive}" "{destination_zip_path_colab}"

# Vérifions sa présence :
print("\nContenu de /content après la copie :")
!ls -lh /content

### 1.C Décompresser le dataset Roboflow
Le ZIP de Roboflow contient déjà la structure `train/`, `valid/` et le fichier data.yaml. Nous allons le décompresser dans un dossier spécifique, `/content/data`.

In [ ]:
!unzip -q /content/data.zip -d /content/data
# Vérifions le contenu de custom_data
!ls /content/data

*(Vous devriez voir `train/`, `valid/`, `test/` et `data.yaml` listés.)*


## ÉTAPE 2 : ENTRAÎNER LE MODÈLE YOLO
C'est le moment clé ! Nous allons lancer l'entraînement.

Ultralytics est la bibliothèque Python qui nous fournit les outils pour entraîner notre modèle YOLO.
Installons la dernière version :

In [ ]:
!pip install ultralytics


### 4.1 Choix des paramètres d'entraînement (À AJUSTER SELON VOS BESOINS)
-   `model`: Le modèle pré-entraîné qui servira de base. Utiliser un modèle pré-entraîné (sur de grands datasets comme COCO) permet un apprentissage plus rapide et efficace (transfer learning).
    *   Exemples : `yolo11n.pt` (nano, le plus petit et rapide, idéal pour commencer), `yolo11s.pt` (small), `yolo11m.pt` (medium), `yolo11l.pt` (large), `yolo11xl.pt` (extra-large, plus précis mais plus lent et gourmand en ressources).
    *   Si vous débutez ou avez un petit dataset, `yolo11n.pt` ou `yolo11s.pt` sont de bons choix.
    *   Vous pouvez aussi utiliser les modèles YOLOv5 et YOLOv8 en changeant `yolo11` par `yolov5` ou `yolov8`.
-   `data`: Chemin vers votre fichier `dofus_data.yaml` créé à l'étape précédente.
-   `epochs`: Nombre de fois que le modèle verra l'ensemble de votre dataset d'entraînement.
    Un bon point de départ est entre 25 et 100. Plus de données ou des tâches plus complexes peuvent nécessiter plus d'époques. Commencez petit (ex: 30-50) pour un premier test.
-   `imgsz` (ou `img`): La taille (résolution en pixels) à laquelle les images seront redimensionnées avant d'être présentées au modèle.
    *   Exemple : `640` (pour des images carrées de 640x640). Des tailles plus grandes peuvent améliorer la détection de petits objets, mais augmentent l'utilisation de mémoire GPU (VRAM) et le temps d'entraînement. `320` est aussi une option pour des entraînements plus rapides.
-   `batch`: Nombre d'images traitées en une seule fois par le GPU.
    Une valeur de `-1` laisse Ultralytics choisir automatiquement la meilleure taille de batch en fonction de la VRAM disponible. Sinon, essayez 8, 16, 32...
    Si vous avez une erreur "CUDA out of memory", réduisez cette valeur ou `imgsz`.
-   `patience`: Nombre d'époques à attendre sans amélioration de la performance sur le set de validation avant d'arrêter l'entraînement prématurément (early stopping). Utile pour éviter le sur-apprentissage. Exemple : `patience=10` (arrête si pas d'amélioration après 10 époques).

Consultez la documentation Ultralytics pour plus d'options : [https://docs.ultralytics.com/modes/train/](https://docs.ultralytics.com/modes/train/)

In [ ]:
# Configuration pour l'entraînement (MODIFIEZ CES VALEURS)
MODEL_CHOICE = 'yolo11s.pt'
NUM_EPOCHS = 100 # Commencez par un nombre plus petit pour tester (ex: 10-25), puis augmentez.
IMAGE_SIZE = 640
BATCH_SIZE = -1 # -1 pour auto-batch, ou un nombre comme 8, 16
PATIENCE_EPOCHS = 10

# Commande d'entraînement
!yolo detect train \
    model={MODEL_CHOICE} \
    data={path_to_output_yaml} \
    epochs={NUM_EPOCHS} \
    imgsz={IMAGE_SIZE} \
    batch={BATCH_SIZE} \
    patience={PATIENCE_EPOCHS}

L'entraînement peut prendre un certain temps (de quelques minutes à plusieurs heures) en fonction de :
- La taille de votre dataset.
- Le modèle (`MODEL_CHOICE`) choisi.
- Le nombre d'époques (`NUM_EPOCHS`).
- La puissance du GPU Colab qui vous est alloué.


**Où trouver les résultats ?**
Les résultats, incluant les poids du modèle entraîné (notamment `best.pt` qui est le plus important), les graphiques de performance, et des exemples de validation, seront sauvegardés dans un dossier. Avec les paramètres ci-dessus, ce sera quelque chose comme :
`/content/runs/detect`
Vous y trouverez un sous-dossier `weights/` contenant `best.pt` (le modèle ayant eu les meilleures performances sur l'ensemble de validation) et `last.pt` (le modèle à la toute dernière époque).

## ÉTAPE 5 : TESTER LE MODÈLE ENTRAÎNÉ
Une fois l'entraînement terminé, il est bon de vérifier visuellement comment votre modèle se comporte sur quelques images qu'il n'a pas vues pendant l'entraînement (celles de votre ensemble de validation).
Nous utiliserons le modèle `best.pt` car c'est généralement celui qui offre les meilleures performances de généralisation


In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

Le modèle devrait dessiner des boîtes (bounding boxes) autour des objets d'intérêt qu'il a appris à reconnaître, avec le nom de la classe et un score de confiance.

**Si les résultats ne sont pas satisfaisants :**
-   **Vérifiez vos annotations :** Des erreurs ou incohérences dans l'étiquetage (dataset) sont la cause la plus fréquente de mauvaises performances.
-   **Augmentez le nombre d'époques :** Le modèle n'a peut-être pas eu assez de temps pour apprendre.
-   **Utilisez un modèle de base plus grand :** Passez à `yolo11l.pt` (cela demandera plus de ressources et de temps).
-   **Augmentez la taille de votre dataset :** Plus d'images, et surtout plus de diversité dans les images (différents angles, éclairages, contextes) améliorent la robustesse du modèle.
-   **Data Augmentation :** Ultralytics applique déjà des techniques d'augmentation de données (rotations, changements de couleurs, etc.) pendant l'entraînement. Vous pouvez explorer des configurations plus avancées si besoin.
-   **Ajustez `imgsz` :** Si vos objets sont petits, une résolution plus élevée peut aider.

Détecter des ressources ici c'est pas mal, mais ce n'est pas très utile pour récolter. On va exporter le modèle pour pouvoir l'utiliser en local sur notre ordinateur.

### VISUALISER LES PERFORMANCES DU MODÈLE
Après l'entraînement, Ultralytics sauvegarde plusieurs graphiques qui nous aident à comprendre si notre modèle est performant. Nous allons afficher les plus importants et expliquer comment les lire, même sans être un expert en IA !

Ces graphiques se trouvent dans le dossier de résultats de votre entraînement.

In [ ]:
import os
import glob
from pathlib import Path # Utilisé pour une recherche plus robuste des dossiers
from IPython.display import Image, display, Markdown
import textwrap

results_base_dir = Path('/content/runs/detect')
latest_experiment_path = None # Sera un str

if results_base_dir.is_dir():
    # Obtenir tous les sous-dossiers commençant par 'train'
    experiment_dirs = [d for d in results_base_dir.glob('train*') if d.is_dir()]

    if experiment_dirs:
        # Trouver celui avec le temps de modification le plus récent
        # Cela correspond généralement au dernier entraînement effectué.
        latest_experiment_path_obj = max(experiment_dirs, key=lambda p: p.stat().st_mtime)
        latest_experiment_path = str(latest_experiment_path_obj) # Convertir Path en string
        print(f"Analyse des résultats depuis : {latest_experiment_path}\n")
    else:
        print(f"Aucun sous-dossier 'train*' trouvé dans '{results_base_dir}'.")
        print("Veuillez vous assurer que l'Étape 4 (Entraînement) a bien été exécutée.")
else:
    print(f"Le dossier de base des résultats '{results_base_dir}' n'existe pas.")
    print("Veuillez vous assurer que l'Étape 4 (Entraînement) a bien été exécutée.")


# --- Suite du code de visualisation, seulement si latest_experiment_path a été trouvé ---
if latest_experiment_path and os.path.isdir(latest_experiment_path):

    # --- 1. Affichage du graphique principal des résultats (results.png) ---
    results_png_path = None # Initialisation
    # Chercher les fichiers PNG pertinents pour les métriques
    possible_results_plots = ['results.png', 'metrics_plot.png', 'P_R_F1_curve.png', 'F1_curve.png', 'PR_curve.png']
    found_results_plot = False
    for plot_name in possible_results_plots:
        potential_path = os.path.join(latest_experiment_path, plot_name)
        if os.path.exists(potential_path):
            results_png_path = potential_path
            found_results_plot = True
            print(f"Graphique des métriques principales trouvé : {results_png_path}")
            break

    if found_results_plot and results_png_path:
        display(Markdown("### 1. Graphique des Métriques d'Entraînement et de Validation"))
        display(Image(filename=results_png_path, width=800)) # Ajustez width si besoin

        explanation_markdown = textwrap.dedent("""
        **Comment lire ce graphique ?**

        *   **Axe X (Horizontal - Epochs) :** Représente les cycles d'entraînement. Plus on va vers la droite, plus le modèle a été entraîné longtemps.
        *   **Axe Y (Vertical) :** Représente la valeur de la métrique (généralement entre 0 et 1, où 1 est parfait).

        **Les courbes importantes à regarder :**
        (Les noms exacts des courbes peuvent légèrement varier, mais cherchez des termes similaires)

        *   **`metrics/precision` (ou P) : Précision.**
            *   **Ce que ça veut dire :** Quand le modèle dit "J'ai trouvé un objet X", quelle est la probabilité qu'il ait raison ?
            *   **Ce qu'on veut :** Une courbe qui monte et se stabilise le plus haut possible (proche de 1).
        *   **`metrics/recall` (ou R) : Rappel (ou Sensibilité).**
            *   **Ce que ça veut dire :** Parmi tous les objets X réellement présents dans les images, combien le modèle a-t-il réussi à trouver ?
            *   **Ce qu'on veut :** Une courbe qui monte et se stabilise le plus haut possible (proche de 1).
        *   **`metrics/mAP50` (ou `mAP@.50`) : Mean Average Precision à IoU=0.5.**
            *   **Ce que ça veut dire :** C'est une sorte de **score global de performance**. Il combine la précision et le rappel pour toutes les classes, et considère une détection comme correcte si la boîte prédite chevauche la vraie boîte d'au moins 50% (c'est le "IoU=0.5").
            *   **Ce qu'on veut :** Une courbe qui monte et se stabilise le plus haut possible (proche de 1). C'est souvent la métrique principale pour juger la qualité globale.
        *   **`metrics/mAP50-95` (ou `mAP@.50:.95`) : mAP moyennée sur plusieurs seuils d'IoU (de 0.5 à 0.95).**
            *   **Ce que ça veut dire :** Similaire à `mAP50`, mais plus strict car il exige un meilleur placement des boîtes sur une plage de seuils.
            *   **Ce qu'on veut :** Une courbe qui monte. Elle sera généralement plus basse que `mAP50`.

        **Courbes `train/...` vs `val/...` :**
        *   Les courbes avec `train` (ex: `train/box_loss`) montrent comment le modèle performe sur les données qu'il *voit* pendant l'entraînement.
        *   Les courbes avec `val` (ex: `val/box_loss` ou directement les métriques comme `metrics/mAP50` qui sont calculées sur le set de validation) montrent comment le modèle performe sur des données *nouvelles* qu'il n'a jamais vues. C'est **CRUCIAL** pour voir si le modèle généralise bien ou s'il "apprend par cœur" (sur-apprentissage).
        *   Idéalement, les courbes `train` et `val` pour les métriques (P, R, mAP) doivent être proches et monter ensemble. Si les performances sur `train` sont excellentes mais celles sur `val` sont bien moins bonnes et stagnent ou baissent, c'est un signe de sur-apprentissage.

        **En résumé pour un non-expert :** Recherchez les courbes de **Précision, Rappel, et mAP50 (ou mAP@0.5)**. Si elles montent et atteignent des valeurs élevées (disons, > 0.7 ou 0.8 pour un bon modèle, mais cela dépend de la difficulté de la tâche), c'est un bon signe ! Si elles stagnent bas, le modèle n'apprend pas bien.
        """)
        display(Markdown(explanation_markdown))
    else:
        print("Le graphique principal des résultats (`results.png` ou similaire) n'a pas été trouvé dans le dossier d'expérience.")

    # --- 2. Affichage de la Matrice de Confusion (confusion_matrix.png) ---
    confusion_matrix_png_path = os.path.join(latest_experiment_path, 'confusion_matrix.png')
    if os.path.exists(confusion_matrix_png_path):
        display(Markdown("### 2. Matrice de Confusion"))
        display(Image(filename=confusion_matrix_png_path, width=600)) # Ajustez width si besoin

        # Essayer de charger classes.txt pour rendre la matrice de confusion plus lisible
        classes_txt_path = '/content/custom_data/classes.txt' # Chemin vers le fichier classes.txt original
        class_names = []
        if os.path.exists(classes_txt_path):
            with open(classes_txt_path, 'r') as f:
                class_names = [line.strip() for line in f if line.strip()]

        explanation_text_cm = textwrap.dedent("""
        **Comment lire cette matrice ?**

        Cette matrice vous montre quelles classes le modèle a tendance à confondre entre elles.
        *   **Lignes :** Les vraies classes des objets (Vérité Terrain).
        *   **Colonnes :** Les classes prédites par le modèle.
        *   **Diagonale (du haut-gauche au bas-droite) :** Les prédictions correctes. Plus les chiffres (et les couleurs plus intenses) sont sur la diagonale, mieux c'est !
        *   **Cases hors diagonale :** Les erreurs de classification. Par exemple, si dans la ligne "Frene" et la colonne "Ble", il y a un chiffre, cela signifie que le modèle a parfois confondu un "Frene" avec un "Ble".
        *   **Dernière ligne/colonne (`background`) :**
            *   `background` en ligne : Indique les objets réels qu'aucune boîte n'a détectés (faux négatifs pour cette classe).
            *   `background` en colonne : Indique les boîtes prédites qui ne correspondaient à aucun objet réel (faux positifs).

        **Ce qu'on veut :**
        *   Des valeurs élevées et des couleurs vives sur la **diagonale principale**.
        *   Des valeurs faibles (proches de 0) et des couleurs claires **partout ailleurs**.
        *   Des valeurs faibles dans la dernière ligne/colonne "background" (sauf pour la case background/background si elle existe).
        """)
        if class_names:
            # Concaténation de chaînes, textwrap.dedent n'est pas nécessaire ici car les f-strings gèrent bien
            class_list_md = "\n**Vos classes sont (dans l'ordre de la matrice, en commençant par 0) :**\n"
            for i, name in enumerate(class_names):
                class_list_md += f"  {i}: {name}\n"
            explanation_text_cm += class_list_md # Ajouter la liste des classes à la fin

        display(Markdown(explanation_text_cm))

    else:
        print(f"La matrice de confusion (`confusion_matrix.png`) n'a pas été trouvée dans {latest_experiment_path}.")


## ÉTAPE 6 : PRÉPARER ET TÉLÉCHARGER VOTRE MODÈLE ENTRAÎNÉ
Nous allons maintenant créer un package ZIP contenant :
1.  Votre meilleur modèle entraîné (`best.pt` renommé en `my_model.pt`).
2.  (Optionnel) Le dossier complet des résultats de l'entraînement.

In [ ]:
# Créer un fichier "dofus_AI" pour y mettre les poids et les résultats d'entrainements
!mkdir /content/dofus_AI
!cp /content/runs/detect/train/weights/best.pt /content/dofus_AI/my_model.pt
!cp -r /content/runs/detect/train /content/dofus_AI

# créer le zip "dofus_AI.zip"
%cd dofus_AI
!zip /content/dofus_AI.zip my_model.pt
!zip -r /content/dofus_AI.zip train
%cd /content

In [ ]:
from google.colab import files

files.download('/content/dofus_AI.zip')

Une fois le téléchargement terminé, vous aurez `dofus_AI.zip` sur votre ordinateur. Ce ZIP contient tout ce dont vous avez besoin pour réutiliser votre modèle ou analyser plus en détail l'entraînement.

**Félicitations !** Vous avez entraîné et packagé votre propre modèle de détection d'objets pour Dofus.